In [1]:
%matplotlib widget
import numpy as np
# from numpy import sin, cos, pi
from numpy import pi
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

class Matrix:
    
    __array_ufunc__ = None
    
    def __init__(self, val):
        self.val = val
        
    def __mul__(self, b):
        if isinstance(b, Matrix):
            val = np.matmul(self.val, b.val)
        else:
            val = np.matmul(self.val, b)
        return Matrix(val)
    
    def __rmul__(self, a):
        if isinstance(a, Matrix):
            val = np.matmul(a.val, self.val)
        else:
            val = np.matmul(a, self.val)
        return Matrix(val)

class Rx(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[0,0] = 1
        rot[1,:] = [0, np.cos(q), -np.sin(q),0]
        rot[2,:] = [0, np.sin(q), np.cos(q),0]
        self.val = rot

class Ry(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[:3,:3] = [[np.cos(q), 0, np.sin(q)],
         [0, 1, 0],
         [-np.sin(q), 0, np.cos(q)]
        ]
        self.val = rot
        
class Rz(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[0,:] = [np.cos(q), -np.sin(q),0,0]
        rot[1,:] = [np.sin(q), np.cos(q),0,0]
        rot[2,2] = 1
        self.val = rot

class T(Matrix):
    def __init__(self, vector):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[0:3,3] = vector
        np.fill_diagonal(mat,1)  
        self.val = mat

class Tx(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[0,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

class Ty(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[1,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

class Tz(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[2,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

In [2]:
class Robot(widgets.HBox):
    
    def extract_plot_points(serial): 
        xs, ys, zs = [],[],[]
        for trans in serial: 
            x,y,z = trans.val[0:3,3]
            xs.append(x)
            ys.append(y)
            zs.append(z)

        return xs,ys,zs


    def extract_vectors_from_trans(trans):
        x,y,z = trans[0:3,3]
        p = [x,y,z]
        v1 = trans[0:3,0]
        v2 = trans[0:3,1]
        v3 = trans[0:3,2]

        return p, [v1,v2,v3]

    def plot_arrow(ax, p,v,color):
        x,y,z = p 
        u,v,w = v 
        ax.quiver(x, y, z, u, v, w, length=2, normalize=True,color=color)
    
    def plot_frame(ax, trans): 
        position, rotations = Robot.extract_vectors_from_trans(trans.val)
        colors = ['r', 'g', 'b']
        for i in range(3): 
            Robot.plot_arrow(ax, position, rotations[i], colors[i])

            
    def update_q1(self, q1):
        self.R1 = Rz(q1.new)
        self.draw() 
 
    def update_d2(self, d2):
        self.T2 = Tz(self.L1 + d2.new)
        self.draw()
        
    def update_d3(self, d3):
        self.T3 = Tx(d3.new)
        self.draw()

    def update_q4(self, q4):
        self.R4 = Rx(q4.new)
        self.draw()

    def update_q5(self, q5):
        self.R5 = Rz(q5.new)
        self.draw()
        
    def update_q6(self, q6):
        self.R6 = Rx(q6.new)
        self.draw()

        
    def __init__(self):
        super().__init__()
        
        self.L1, L2, L3, L4 = 30, 10, 5, 5
        q1, d2, d3, q4, q5, q6 = 0, 5, 5, 0, 0, 0
        
        self.R1 = Rz(q1)
        self.T2 = Tz(self.L1 + d2)
        self.T3 = Tx(d3)
        self.R4 = Rx(q4)
        self.T4 = Tx(L2)
        self.R5 = Rz(q5)
        self.T5 = Tx(L3)
        self.R6 = Rx(q6)
        self.T6 = Tx(L4)

        output = widgets.Output()
        
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(6.5, 6.5))
            self.ax = self.fig.add_subplot(111, projection='3d')
        
        
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(True)

        q1_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.1, value=0, description='q1')
        d2_slider = widgets.FloatSlider(min=0, max=20, step=0.5, value=5, description='d2')
        d3_slider = widgets.FloatSlider(min=0, max=20, step=0.5, value=5, description='d3')
        q4_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.1, value=0, description='q4')
        q5_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.1, value=0, description='q5')
        q6_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.1, value=0, description='q6')
        
        
        q1_slider.observe(self.update_q1, 'value')
        d2_slider.observe(self.update_d2, 'value')
        d3_slider.observe(self.update_d3, 'value')
        q4_slider.observe(self.update_q4, 'value')
        q5_slider.observe(self.update_q5, 'value')
        q6_slider.observe(self.update_q6, 'value')
        

        controls = widgets.VBox([
            q1_slider, d2_slider, d3_slider, q4_slider, q5_slider, q6_slider,
        ])
        controls.layout = widgets.Layout(
            display='flex',
            flex='flex-shrink'
         )
        
        out_box = widgets.Box([output])
        output.layout = widgets.Layout(
            display='flex',
            flex='flex-shrink'
         )

        self.children = [controls, output]
        self.draw()
        
    def draw(self):
        plt.cla()
        
        A1 = self.R1
        A2 = self.T2
        A3 = self.T3 * self.R4
        A4 = self.T4 * self.R5
        A5 = self.T5 * self.R6
        A6 = self.T6
        
        A = [A2, A3, A4, A5, A6]
        states = [A1]
        for a in A:
            states.append(states[-1] * a)
        
        for state in states:
            Robot.plot_frame(self.ax, state)

        xs,ys,zs = Robot.extract_plot_points(states)
        self.ax.plot(xs,ys,zs, linewidth=2)      
        
        self.ax.set_xlim3d([-10,30])
        self.ax.set_ylim3d([-10,30])
        self.ax.set_zlim3d([-10,30])
        plt.show() 
    
        
Robot()

Robot(children=(VBox(children=(FloatSlider(value=0.0, description='q1', max=6.283185307179586), FloatSlider(va…